# Question 1: Analyse des débits avec MIMO

Interprétation :
* Le débit n'est pas exactement doublé (+77.8% au lieu de +100%)
* Écart dû aux overheads du protocole (ACK, beacons, SIFS/DIFS)
* Performance réelle proche du maximum théorique

### 1 flux spatial : PDR = 56.2% (PDR = Packet Delivery Ratio)

* Trafic envoyé : ~118 Mbps (900,000 paquets × 1472 bytes / 10s)
* Capacité : ~67 Mbps
* Résultat : Saturation → Pertes importantes

### 2 flux spatiaux : PDR = 99.97%
* Trafic envoyé : ~118 Mbps
* Capacité : ~120 Mbps
* Résultat : Capacité suffisante → Quasi aucune perte

### Explication de l'augmentation du débit:

1. *Principe du multiplexage spatial*

- Deux flux indépendants transmis simultanément
- Même bande passante (20 MHz)
- Même fréquence utilisée
- Débit quasi-doublé sans augmenter la largeur du canal

2. *Exploitation des trajets multiples*

#### Sans MIMO (SISO) :
* Les trajets multiples causent des interférences destructives
* Fading sélectif en fréquence
* Performance dégradée

#### Avec MIMO (2×2) :
* Les trajets multiples sont exploités positivement: si un trajet est affaibli, l'autre peut compenser
* Chaque trajet transporte un flux différent
* Les antennes multiples séparent les flux
* Diversité → Robustesse accrue


# Question 2: Impact de la distance et du canal

## Distance : 5m -> 50m

Dans notre simulation, le débit reste remarquablement stable (≈ 132.7 Mbps) de 5 m à 20 m, puis commence à décroître à 30 m et chute fortement à 40–50 m.

Cette stabilité initiale s’explique par le fait que, dans cette plage de distances, le rapport signal-sur-bruit (SNR) reçu reste supérieur au seuil requis pour le MCS (Modulation and Coding Scheme) utilisé  HtMcs15 :

HtMcs15 correspond à :
* 2 flux spatiaux,
* modulation 64-QAM,
* taux de codage 5/6,
* canal 20 MHz,
* GI (Guard Interval) de 800 ns.

Il nécessite un SNR minimal d’environ 23–25 dB pour un taux d’erreur binaire (BER) acceptable (~10⁻⁵).
👉 Tant que le SNR ≥ 25 dB (typiquement jusqu’à ~20–25 m en indoor avec des antennes standard), toutes les trames sont correctement décodées → PDR ≈ 100 %, débit constant.

Mais à 30 m, l’atténuation du signal (loi en d⁻ⁿ, n ≈ 3–4 en indoor) fait descendre le SNR sous le seuil critique (ex. < 22 dB). 

*Conséquences :*
- Erreurs de décodage apparaissent (surtout sur les symboles 64-QAM les plus sensibles),
- Les trames sont perdues (non acquittées par ACK) → PDR diminue,
- Comme nous utilisons ConstantRateWifiManager, le STA ne rétrograde pas vers un MCS plus robuste (ex. HtMcs7, 16-QAM 3/4) : il continue à émettre à HtMcs15, mais avec un taux d’échec croissant,

==> *Résultat :* le débit utile (goodput) chute, non pas parce que la couche physique ralentit, mais parce que de plus en plus de paquets doivent être retransmis ou sont définitivement perdus.

*Illustration concrète :*
À 50 m, un PDR de ~50 % signifierait que seulement la moitié des paquets atteignent la destination — donc, même si la PHY tente d’émettre à 300 Mbps (débit brut théorique de HtMcs15), le débit utile (goodput) est divisé par deux (plus pertes liées aux retransmissions et overhead), d’où les ~65 Mbps observés. 

## Canal : 20 vs 40MHz

### ✅ 1. À courte distance (≤ 20 m) : gain élevé (~87 %, soit ×1.87)
À faible distance, le rapport signal-sur-bruit (SNR) est élevé (ex. > 25 dB). Dans ce régime :
- Le récepteur peut exploiter des modulations d’ordre élevé (64-QAM) et des codes correcteurs peu redondants (taux de codage 5/6), même avec deux flux spatiaux.
- Passer de 20 MHz à 40 MHz double quasiment le nombre de sous-porteuses utiles (de 56 à 114 en 802.11n HT), donc double la capacité théorique.

Le gain mesuré est de ×1.87 (et non ×2.0) à cause de :
- L’overhead MAC (trames de gestion, ACK, SIFS/DIFS, contention),
- La guard interval (800 ns ou 400 ns),
- Les pertes de paquets résiduelles (bien que très faibles ici, PDR ≈ 100 %).
→ C’est donc un gain quasi-optimal, conforme aux limites physiques du système.

### ⚠️ 2. À grande distance (≥ 30 m) : gain réduit (×1.3 à ×1.4)
Lorsque la distance augmente : La puissance reçue diminue selon la loi de propagation (ex. ∝ 1/d² ou 1/d³), ce qui réduit le SNR.
Or, le bruit thermique dans le récepteur est proportionnel à la largeur de bande : N=kTB⇒Si B double (20 → 40 MHz), N augmente de 10log10(2)≈+3dB

→ Pour une même puissance émise, le SNR baisse de 3 dB en 40 MHz par rapport à 20 MHz.

Conséquence :
*Le SNR en 40 MHz atteint plus rapidement le seuil minimal requis pour un MCS donné.*

**Exemple :**

À 30 m, le SNR en 20 MHz permet encore HtMcs14 (2 streams, 64-QAM 3/4, ~130 Mbps PHY),
Mais en 40 MHz, le SNR −3 dB devient insuffisant → le lien tente de garder le même MCS (ici, gestion fixe), mais subit plus d’erreurs → PER augmente, retransmissions → bon débit (goodput) chute plus vite.
→ Résultat : le débit en 40 MHz progresse moins vite que celui en 20 MHz avec la distance, donc le rapport 40 MHz / 20 MHz diminue (de ×1.87 à ×1.34 entre 5 m et 30 m).

### 📌 En résumé :

Le channel bonding augmente la capacité, mais aussi la sensibilité au bruit. Son efficacité dépend donc fortement du SNR disponible — excellent en conditions favorables (courte portée), mais pénalisé en conditions dégradées (grande distance, interférences). 


## Analyse combinée

Les résultats montrent clairement que le MIMO 2×2 + channel bonding est hautement performant, mais conditionnellement :

✅ *Optimal à courte portée (≤ 20 m) :* Débit > 240 Mbps, PDR ≈ 100 % → configuration idéale pour le high-throughput indoor (ex. streaming 4K, transfert de fichiers).

⚠️ *Dégradation progressive au-delà de 30 m :* Le débit chute plus vite en 40 MHz qu’en 20 MHz (ex. −36 % vs −51 % entre 5 m et 50 m en 40 MHz, mais pente plus raide entre 20 m et 40 m). Cela indique que le 40 MHz est plus sensible à la distance — un compromis classique entre capacité et portée.

🔍 *Interprétation physique :*
Le MIMO exploite la richesse du canal (multi-trajets) pour séparer les flux. À courte distance, l’environnement indoor fournit suffisamment de scattering. À plus grande distance, l’atténuation domine, les canaux deviennent corrélés, et le gain spatial s’effondre. Le channel bonding, en élargissant la bande, amplifie ce phénomène via une sensibilité accrue au fading sélectif en fréquence.